## Simple Collision avoidance

In [1]:
import numpy as np

# Custom modules

from helpers import global2local, plot_3d_interactive, kill_processes
from simulators import Simulator, Gazebo, QGC
from plan import Plan, State
from env import Enviroment

kill_processes()

## Create Plans

In [2]:
offsets = [(-10, 0, 0, 0),(10, 0, 0, 0)]#
n_vehicles=len(offsets)


homes=np.array([offset[:3] for offset in offsets])
global_paths=[np.array([[-10, 0, 5],[10, 0, 5]]),
              np.array([[10, 0, 5],[-10, 0, 5]])]
local_paths=[global2local(path, home,pairwise=True) for path,home in zip(global_paths,homes)]
plans=[Plan.basic(wps=path,alt=5,wp_margin=0.5,navegation_speed=1) for path in local_paths]

## Visualization Parameters

In [3]:
waypoints = {f'waypoint_{i+1}': {'pos':global_paths[0],'color':'blue'} for i in range(n_vehicles)}
plot_3d_interactive(waypoints,title='Simulation Markers',expand=[0.2,0.2,0.6],ground=-0.05)

## Choose Simulator

In [4]:
# simulator = Simulator(name="NONE",offsets=offsets,plans=plans)

# simulator=QGC(offsets=offsets,
#               plans=plans,
#               origin=(-35.3633245,149.1652241,0,0))

simulator=Gazebo(offsets=offsets,
                 plans=plans,
                 world_path='simulators/gazebo/worlds/runway.world',
                 vehicle_models=n_vehicles*['drone'],
                 markers=waypoints)

Launch Simulator

In [5]:
uavs = simulator.launch()
env = Enviroment(uavs)

Vehicle 1 launched 🚀
Vehicle 2 launched 🚀


## Execute Plan

In [6]:
while env.vehs:
    env.gather_broadcasts()
    for uav in env.vehs.copy():
        env.update_neighbors(uav)
        if uav.plan.state == State.DONE:
            env.remove(uav)
        else:
            uav.act()
kill_processes()

Vehicle 1: ▶️ Starting Step: Check disarmed
Vehicle 2: ▶️ Starting Step: Check disarmed
Vehicle 1: ✅ Step: Check disarmed is done
Vehicle 1: ▶️ Starting Step: Check EKF
Vehicle 2: ✅ Step: Check disarmed is done
Vehicle 2: ▶️ Starting Step: Check EKF
Vehicle 1: ✅ Step: Check EKF is done
Vehicle 1: ▶️ Starting Step: Check GPS
Vehicle 2: ✅ Step: Check EKF is done
Vehicle 2: ▶️ Starting Step: Check GPS
Vehicle 1: ✅ Step: Check GPS is done
Vehicle 1: ▶️ Starting Step: Check system
Vehicle 2: ✅ Step: Check GPS is done
Vehicle 2: ▶️ Starting Step: Check system
Vehicle 1: ✅ Step: Check system is done
Vehicle 1: ▶️ Starting Step: Switch to GUIDED
Vehicle 1: ✅ Step: Switch to GUIDED is done
Vehicle 1: ▶️ Starting Step: Set speed to 1.00 m/s
Vehicle 1: ✅ Step: Set speed to 1.00 m/s is done
Vehicle 1: ▶️ Starting Step: arm
Vehicle 2: ✅ Step: Check system is done
Vehicle 2: ▶️ Starting Step: Switch to GUIDED
Vehicle 2: ✅ Step: Switch to GUIDED is done
Vehicle 2: ▶️ Starting Step: Set speed to 1.00 

In [7]:
uavs[0].plan

✅ <Plan 'basic'>
  ✅ <Action 'PREARM'>
    ✅ <Step 'Check disarmed'>
    ✅ <Step 'Check EKF'>
    ✅ <Step 'Check GPS'>
    ✅ <Step 'Check system'>
  ✅ <Action 'MODE: GUIDED'>
    ✅ <Step 'Switch to GUIDED'>
  ✅ <Action 'PARAM: nav speed'>
    ✅ <Step 'Set speed to 1.00 m/s'>
  ✅ <Action 'ARM'>
    ✅ <Step 'arm'>
  ✅ <Action 'TAKEOFF'>
    ✅ <Step 'takeoff'>
  ✅ <Action 'FLY'>
    ✅ <Step 'go to  -> (0, 0, 5)'>
    ✅ <Step 'go to (avoidance) -> (7.532687582721661, 1.0234097289705013, 4.865453243255615)'>
    ✅ <Step 'go to (avoidance) -> (8.354908181218905, 1.952799056930154, 4.874826908111572)'>
    ✅ <Step 'go to (avoidance) -> (9.367530107534394, 2.699772833730312, 4.882475852966309)'>
    ✅ <Step 'go to (avoidance) -> (10.955369517576035, 2.8755139112637655, 4.89224100112915)'>
    ✅ <Step 'go to (avoidance) -> (12.786857169829208, 2.472796203245862, 4.904823303222656)'>
    ✅ <Step 'go to (avoidance) -> (13.640175309693122, 2.127278009059454, 4.907873630523682)'>
    ✅ <Step 'go to

In [8]:
for step in uavs[1].plan.steps[5].steps:
    print(step.curr_pos)

[0.01128037 0.01183903 4.51926994]
[-7.59709549 -1.0090878   4.91589546]
[-7.98986959 -1.91399419  4.92028856]
[-8.1646452  -2.01246762  4.91749001]
[-8.87658882 -2.56589937  4.91933489]
[-9.32653427 -2.46652961  4.91749859]
[-9.90219975 -2.19146299  4.91609812]
[-9.32216454 -1.35692048  4.91709948]
[-8.29049206 -1.83895791  4.91739893]
[-7.71958685 -1.0997436   4.91382885]
[-19.50762749  -0.0645831    4.99810314]


In [9]:
uavs[1].plan

✅ <Plan 'basic'>
  ✅ <Action 'PREARM'>
    ✅ <Step 'Check disarmed'>
    ✅ <Step 'Check EKF'>
    ✅ <Step 'Check GPS'>
    ✅ <Step 'Check system'>
  ✅ <Action 'MODE: GUIDED'>
    ✅ <Step 'Switch to GUIDED'>
  ✅ <Action 'PARAM: nav speed'>
    ✅ <Step 'Set speed to 1.00 m/s'>
  ✅ <Action 'ARM'>
    ✅ <Step 'arm'>
  ✅ <Action 'TAKEOFF'>
    ✅ <Step 'takeoff'>
  ✅ <Action 'FLY'>
    ✅ <Step 'go to  -> (0, 0, 5)'>
    ✅ <Step 'go to (avoidance) -> (-7.501839079608868, -1.0243559601568912, 4.913533687591553)'>
    ✅ <Step 'go to (avoidance) -> (-8.001973820714754, -2.0124577035137112, 4.916295051574707)'>
    ✅ <Step 'go to (avoidance) -> (-8.166979403223154, -1.912709511216617, 4.916295051574707)'>
    ✅ <Step 'go to (avoidance) -> (-8.95333783973496, -2.626628576091988, 4.917489528656006)'>
    ✅ <Step 'go to (avoidance) -> (-9.422344729673203, -2.493082642571749, 4.917208671569824)'>
    ✅ <Step 'go to (avoidance) -> (-9.987993758879501, -2.1408877388140994, 4.914475917816162)'>
    ✅ <S